In [1]:
# データ処理ライブラリ
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# 画像処理ライブラリ
import cv2

# 可視化ライブラリ
import matplotlib.pyplot as plt

# Keras
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

# Pythonモジュール
import os
import glob

# 表示設定
pd.options.display.max_rows = 78
pd.options.display.max_columns = 78

# pathの設定
path = "./hiragana73/"

# ファイルの読み込み
charfeatures = np.load(path + 'charfeatures.npy')
files_df = pd.read_csv(path + 'files_df.csv')

Using TensorFlow backend.


In [2]:
# ターゲットデータの作成
y = files_df['Label']

In [3]:
# ターゲットデータの前処理
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [4]:
# 平仮名と数字のマッピングデータを作成
encoder_character_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(encoder_character_mapping)

{'あ': 0, 'い': 1, 'う': 2, 'え': 3, 'お': 4, 'か': 5, 'が': 6, 'き': 7, 'ぎ': 8, 'く': 9, 'ぐ': 10, 'け': 11, 'げ': 12, 'こ': 13, 'ご': 14, 'さ': 15, 'ざ': 16, 'し': 17, 'じ': 18, 'す': 19, 'ず': 20, 'せ': 21, 'ぜ': 22, 'そ': 23, 'ぞ': 24, 'た': 25, 'だ': 26, 'ち': 27, 'ぢ': 28, 'つ': 29, 'づ': 30, 'て': 31, 'で': 32, 'と': 33, 'ど': 34, 'な': 35, 'に': 36, 'ぬ': 37, 'ね': 38, 'の': 39, 'は': 40, 'ば': 41, 'ぱ': 42, 'ひ': 43, 'び': 44, 'ぴ': 45, 'ふ': 46, 'ぶ': 47, 'ぷ': 48, 'へ': 49, 'べ': 50, 'ぺ': 51, 'ほ': 52, 'ぼ': 53, 'ぽ': 54, 'ま': 55, 'み': 56, 'む': 57, 'め': 58, 'も': 59, 'や': 60, 'ゆ': 61, 'よ': 62, 'ら': 63, 'り': 64, 'る': 65, 'れ': 66, 'ろ': 67, 'わ': 68, 'ゐ': 69, 'ゑ': 70, 'を': 71, 'ん': 72}


In [5]:
# ラベルエンコーダーで変換
y_encoded = encoder.transform(y)

In [6]:
# 訓練/テストデータ 分割
X_train, X_test, y_train, y_test = train_test_split(charfeatures, y_encoded, train_size=10000, test_size=10000, random_state=42)
y_eval = y_test.copy()

In [7]:
# データをfloat32型へ変換
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255
X_test /= 255

In [8]:
# サイズ変更
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [9]:
# バイナリーのクラス行列へ変換
y_train = np_utils.to_categorical(y_train, 73)
y_test = np_utils.to_categorical(y_test, 73)


# CNN モデル訓練

### model_1
* ベースラインモデルとしてシンプルな構造のCNN
* 畳み込み層 x 2, maxpooling 層 x 2, 全結合層 x 2

### model_2
* model_1 と同等の層を持つ CNN
* model_1 の各層にドロップアウト（0.25）を追加

### model_3
* model_2 をより深くした構造の CNN
* 畳み込み層 x 3, maxpooling 層 x 3, 全結合層 x 2


# model_1の訓練

## 1. 入力層＋畳み込み層の追加

In [10]:
# Sequentialモデルの生成
model_1 = Sequential()

# 入力層兼畳み込み層を追加
model_1.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(48,48,1)))

Instructions for updating:
Colocations handled automatically by placer.


## 2. プール層の追加

In [11]:
# プール層を追加
model_1.add(MaxPooling2D(pool_size=(2, 2)))

## 3. 畳み込み層＋プール層の追加

In [12]:
# 畳み込み層の追加
model_1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# プール層の追加
model_1.add(MaxPooling2D(pool_size=(2, 2)))

## 4. 全結合層 x 2 ＋出力層の追加

In [13]:
# 全結合層
model_1.add(Flatten())

# 全結合層
model_1.add(Dense(128, activation='relu'))

# 出力層
model_1.add(Dense(73, activation='softmax'))

## 5. コンパイル

In [14]:
# モデル コンパイル
model_1.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

## モデルの確認

In [15]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 73)                9417      
Total para

## 訓練中の性能監視を準備する

In [16]:
# EarlyStoppingの設定
from keras.callbacks import EarlyStopping
callbacks = EarlyStopping(monitor='val_loss', patience=3)

## モデルの訓練

In [17]:
# モデル訓練
model_1.fit(X_train, y_train, validation_data = (X_test, y_test), callbacks=[callbacks], epochs=50)

Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 23s 2ms/step - loss: 4.2610 - acc: 0.0209 - val_loss: 4.2285 - val_acc: 0.0121
Epoch 2/50
10000/10000 [==============================] - 22s 2ms/step - loss: 4.0638 - acc: 0.0710 - val_loss: 3.8463 - val_acc: 0.0936
Epoch 3/50
10000/10000 [==============================] - 24s 2ms/step - loss: 2.5684 - acc: 0.3796 - val_loss: 1.3766 - val_acc: 0.6154
Epoch 4/50
10000/10000 [==============================] - 25s 2ms/step - loss: 0.6981 - acc: 0.8427 - val_loss: 0.5612 - val_acc: 0.8575
Epoch 5/50
10000/10000 [==============================] - 24s 2ms/step - loss: 0.3584 - acc: 0.9157 - val_loss: 0.3409 - val_acc: 0.9244
Epoch 6/50
10000/10000 [==============================] - 26s 3ms/step - loss: 0.2565 - acc: 0.9409 - val_loss: 0.3014 - val_acc: 0.9297
Epoch 7/50
10000/10000 [==============================] - 26s 3ms/step - loss: 0

In [18]:
# テストデータで推測
pred_test = model_1.predict_classes(X_test)
pred_test[0:5]

array([19, 52, 27, 63, 18], dtype=int64)

In [19]:
# 正解率の算出
accuracy_score(y_eval, pred_test)

0.9517

- - -
# model_2 の訓練

In [20]:
# model_2の構築
model_2 = Sequential()
model_2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(48,48,1)))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(128, activation='relu'))
model_2.add(Dense(73, activation='softmax'))

# モデル コンパイル
model_2.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# モデルサマリー出力
model_2.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 64)        0         
___________________________

In [21]:
# EarlyStoppingの設定
from keras.callbacks import EarlyStopping
callbacks = EarlyStopping(monitor='val_loss', patience=3)

# モデル訓練
model_2.fit(X_train, y_train, validation_data = (X_test, y_test), callbacks=[callbacks], epochs=50)

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 28s 3ms/step - loss: 4.2618 - acc: 0.0225 - val_loss: 4.2368 - val_acc: 0.0325
Epoch 2/50
10000/10000 [==============================] - 29s 3ms/step - loss: 3.9995 - acc: 0.0702 - val_loss: 3.6280 - val_acc: 0.1192
Epoch 3/50
10000/10000 [==============================] - 29s 3ms/step - loss: 2.3061 - acc: 0.4144 - val_loss: 1.2549 - val_acc: 0.7057
Epoch 4/50
10000/10000 [==============================] - 28s 3ms/step - loss: 0.7793 - acc: 0.8001 - val_loss: 0.5215 - val_acc: 0.8898
Epoch 5/50
10000/10000 [==============================] - 29s 3ms/step - loss: 0.4904 - acc: 0.8767 - val_loss: 0.4210 - val_acc: 0.9064
Epoch 6/50
10000/10000 [==============================] - 30s 3ms/step - loss: 0.3785 - acc: 0.9035 - val_loss: 0.3581 - val_acc: 0.9218
Epoch 7/50
10000/10000 [==============================] - 30s 3ms/step - loss: 0.3160 - acc: 0.9212 - val_loss: 0.2880 - val_acc

In [22]:
# テストデータで推測
pred_test = model_2.predict_classes(X_test)

# 正解率の算出
accuracy_score(y_eval, pred_test)

0.961

# model_3 の訓練

In [23]:
# model_3の構築
model_3 = Sequential()
model_3.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(48,48,1)))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(128, activation='relu'))
model_3.add(Dropout(0.25))
model_3.add(Dense(73, activation='softmax'))

# モデル コンパイル
model_3.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# モデルサマリー出力
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 128)         73856     
__________

In [24]:
# EarlyStoppingの設定
from keras.callbacks import EarlyStopping
callbacks = EarlyStopping(monitor='val_loss', patience=3)

# モデル訓練
model_3.fit(X_train, y_train, validation_data = (X_test, y_test), callbacks=[callbacks], epochs=50)

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 34s 3ms/step - loss: 4.2879 - acc: 0.0166 - val_loss: 4.2838 - val_acc: 0.0189
Epoch 2/50
10000/10000 [==============================] - 37s 4ms/step - loss: 4.2751 - acc: 0.0207 - val_loss: 4.2771 - val_acc: 0.0196
Epoch 3/50
10000/10000 [==============================] - 37s 4ms/step - loss: 4.2637 - acc: 0.0204 - val_loss: 4.2652 - val_acc: 0.0236
Epoch 4/50
10000/10000 [==============================] - 37s 4ms/step - loss: 4.2419 - acc: 0.0290 - val_loss: 4.2405 - val_acc: 0.0321
Epoch 5/50
10000/10000 [==============================] - 37s 4ms/step - loss: 4.1608 - acc: 0.0462 - val_loss: 4.0973 - val_acc: 0.1338
Epoch 6/50
10000/10000 [==============================] - 40s 4ms/step - loss: 3.6724 - acc: 0.1291 - val_loss: 2.9178 - val_acc: 0.4444
Epoch 7/50
10000/10000 [==============================] - 41s 4ms/step - loss: 2.2225 - acc: 0.4149 - val_loss: 1.1902 - val_acc

In [25]:
# テストデータで推測
pred_test = model_3.predict_classes(X_test)

# 正解率の算出
accuracy_score(y_eval, pred_test)

0.9695